In [40]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3
from flask import jsonify
import json
import requests

In [32]:
# read csv & clean data
lifecsv = pd.read_csv("Life Expectancy (years) by Country.csv", encoding='latin1')

lifecsv = lifecsv.drop(columns=['Series Name','Series Code'])

lifecsv = lifecsv.rename(columns={'1999 [YR1999]':'1999','2000 [YR2000]':'2000','2001 [YR2001]':'2001','2002 [YR2002]':'2002','2003 [YR2003]':'2003','2004 [YR2004]':'2004','2005 [YR2005]':'2005','2006 [YR2006]':'2006','2007 [YR2007]':'2007','2008 [YR2008]':'2008','2009 [YR2009]':'2009','2010 [YR2010]':'2010','2011 [YR2011]':'2011','2012 [YR2012]':'2012','2013 [YR2013]':'2013','2014 [YR2014]':'2014','2015 [YR2015]':'2015','2016 [YR2016]':'2016','2017 [YR2017]':'2017'})

lifecsv = lifecsv.dropna()

In [33]:
lifecsv.head()

,Country Name,Country Code,1999,2000,2001,2002,2003,2004,2005,2006,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,AFG,54.959,55.482,56.044,56.637,57.25,57.875,58.5,59.11,...,60.243,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673,64.047
1,Albania,ALB,73.588,73.955,74.286,74.575,74.82,75.028,75.217,75.418,...,75.943,76.281,76.652,77.031,77.389,77.702,77.963,78.174,78.345,78.495
2,Algeria,DZA,69.822,70.292,70.778,71.276,71.78,72.281,72.767,73.225,...,74.032,74.374,74.676,74.944,75.187,75.418,75.641,75.86,76.078,76.293
3,American Samoa,ASM,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Andorra,AND,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [34]:
engine = create_engine("sqlite:///life.sqlite")
conn = engine.connect()

In [35]:
# lifecsv.to_sql(name='life', con=engine, if_exists='replace', index=True)

In [36]:
inspector = inspect(engine)
inspector.get_table_names()

['life']

In [37]:
data = pd.read_sql("SELECT * FROM life", conn)
data.head()

,id,Country_Name,Country_Code,Yr1999,Yr2000,Yr2001,Yr2002,Yr2003,Yr2004,Yr2005,...,Yr2008,Yr2009,Yr2010,Yr2011,Yr2012,Yr2013,Yr2014,Yr2015,Yr2016,Yr2017
0,0,Afghanistan,AFG,54.959,55.482,56.044,56.637,57.25,57.875,58.5,...,60.243,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673,64.047
1,1,Albania,ALB,73.588,73.955,74.286,74.575,74.82,75.028,75.217,...,75.943,76.281,76.652,77.031,77.389,77.702,77.963,78.174,78.345,78.495
2,2,Algeria,DZA,69.822,70.292,70.778,71.276,71.78,72.281,72.767,...,74.032,74.374,74.676,74.944,75.187,75.418,75.641,75.86,76.078,76.293
3,3,American Samoa,ASM,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,4,Andorra,AND,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [10]:
engine = create_engine("sqlite:///life.sqlite")
session = Session(engine)

In [11]:
Base = automap_base()

In [12]:
Base.prepare(engine, reflect=True)

In [13]:
Base.classes.keys()

['life']

In [14]:
columns = inspector.get_columns('life')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
Country_Name TEXT
Country_Code TEXT
Yr1999 FLOAT
Yr2000 FLOAT
Yr2001 FLOAT
Yr2002 FLOAT
Yr2003 FLOAT
Yr2004 FLOAT
Yr2005 FLOAT
Yr2006 FLOAT
Yr2007 FLOAT
Yr2008 FLOAT
Yr2009 FLOAT
Yr2010 FLOAT
Yr2011 FLOAT
Yr2012 FLOAT
Yr2013 FLOAT
Yr2014 FLOAT
Yr2015 FLOAT
Yr2016 FLOAT
Yr2017 FLOAT


In [15]:
Life = Base.classes.life

In [16]:
first_row = session.query(Life).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x19adb4414e0>,
 'Yr1999': 54.959,
 'Yr2015': 63.288,
 'Yr2007': 59.694,
 'Yr2000': 55.482,
 'Yr2016': 63.673,
 'Yr2008': 60.243,
 'Yr2001': 56.044,
 'Yr2017': 64.047,
 'Yr2009': 60.754,
 'Yr2002': 56.637,
 'Yr2010': 61.226,
 'Yr2003': 57.25,
 'Yr2011': 61.666,
 'Yr2004': 57.875,
 'id': 0,
 'Yr2012': 62.086,
 'Yr2005': 58.5,
 'Country_Name': 'Afghanistan',
 'Yr2013': 62.494,
 'Yr2006': 59.11,
 'Country_Code': 'AFG',
 'Yr2014': 62.895}

In [25]:
for row in session.query(Life.Country_Name, Life.Yr1999, Life.Yr2017).limit(15).all():
    print(row)

('Afghanistan', 54.959, 64.047)
('Albania', 73.588, 78.495)
('Algeria', 69.822, 76.293)
('American Samoa', '..', '..')
('Andorra', '..', '..')
('Angola', 46.105, 61.809)
('Antigua and Barbuda', 73.327, 76.519)
('Argentina', 73.618, 76.738)
('Armenia', 70.944, 74.782)
('Aruba', 73.738, 76.01)
('Australia', 78.93170732, 82.49756098)
('Austria', 77.87560976, 81.64146341)
('Azerbaijan', 66.491, 72.123)
('Bahamas, The', 72.093, 75.823)
('Bahrain', 74.239, 77.038)


In [30]:
results = session.query(Life.Country_Name, Life.Country_Code, Life.Yr1999, Life.Yr2000,Life.Yr2001,Life.Yr2002,Life.Yr2003,Life.Yr2004,Life.Yr2005,Life.Yr2006,Life.Yr2007,Life.Yr2008,Life.Yr2009,Life.Yr2010,Life.Yr2011,Life.Yr2012,Life.Yr2013,Life.Yr2014,Life.Yr2015,Life.Yr2016,Life.Yr2017).all()

country_data = []
for name, code, y99, y00, y01, y02, y03, y04, y05, y06,y07,y08,y09,y10,y11,y12,y13,y14,y15,y16,y17 in results:
    country_dict = {}
    country_dict['Country Name'] = name
    country_dict['Country Code'] = code
    country_dict['1999'] = y99
    country_dict['2000'] = y00
    country_dict['2001'] = y01
    country_dict['2002'] = y02
    country_dict['2003'] = y03
    country_dict['2004'] = y04
    country_dict['2005'] = y05
    country_dict['2006'] = y06
    country_dict['2007'] = y07
    country_dict['2008'] = y08
    country_dict['2009'] = y09
    country_dict['2010'] = y10
    country_dict['2011'] = y11
    country_dict['2012'] = y12
    country_dict['2013'] = y13
    country_dict['2014'] = y14
    country_dict['2015'] = y15
    country_dict['2016'] = y16
    country_dict['2017'] = y17
    country_data.append(country_dict)

In [39]:
country_codes = []
for row in data['Country_Code']:
    country_codes.append(row)
country_codes

['AFG',
 'ALB',
 'DZA',
 'ASM',
 'AND',
 'AGO',
 'ATG',
 'ARG',
 'ARM',
 'ABW',
 'AUS',
 'AUT',
 'AZE',
 'BHS',
 'BHR',
 'BGD',
 'BRB',
 'BLR',
 'BEL',
 'BLZ',
 'BEN',
 'BMU',
 'BTN',
 'BOL',
 'BIH',
 'BWA',
 'BRA',
 'VGB',
 'BRN',
 'BGR',
 'BFA',
 'BDI',
 'CPV',
 'KHM',
 'CMR',
 'CAN',
 'CYM',
 'CAF',
 'TCD',
 'CHI',
 'CHL',
 'CHN',
 'COL',
 'COM',
 'COD',
 'COG',
 'CRI',
 'CIV',
 'HRV',
 'CUB',
 'CUW',
 'CYP',
 'CZE',
 'DNK',
 'DJI',
 'DMA',
 'DOM',
 'ECU',
 'EGY',
 'SLV',
 'GNQ',
 'ERI',
 'EST',
 'SWZ',
 'ETH',
 'FRO',
 'FJI',
 'FIN',
 'FRA',
 'PYF',
 'GAB',
 'GMB',
 'GEO',
 'DEU',
 'GHA',
 'GIB',
 'GRC',
 'GRL',
 'GRD',
 'GUM',
 'GTM',
 'GIN',
 'GNB',
 'GUY',
 'HTI',
 'HND',
 'HKG',
 'HUN',
 'ISL',
 'IND',
 'IDN',
 'IRN',
 'IRQ',
 'IRL',
 'IMN',
 'ISR',
 'ITA',
 'JAM',
 'JPN',
 'JOR',
 'KAZ',
 'KEN',
 'KIR',
 'PRK',
 'KOR',
 'XKX',
 'KWT',
 'KGZ',
 'LAO',
 'LVA',
 'LBN',
 'LSO',
 'LBR',
 'LBY',
 'LIE',
 'LTU',
 'LUX',
 'MAC',
 'MDG',
 'MWI',
 'MYS',
 'MDV',
 'MLI',
 'MLT',
 'MHL',


In [56]:
country_names = []
for row in data['Country_Name']:
    country_names.append(row)
country_names

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',

In [59]:
count = 0

income_list = []

for code in country_codes:
    country_code = code
    url = 'http://api.worldbank.org/v2/country/'+ country_code + '?format=json'
    response = requests.get(url).json()
    income_dict = {}
    income_dict['Country_Name'] = country_names[count]
    if response[1][0]['incomeLevel']['value'] != 'Aggregates':
        income_dict['Income_Level'] = response[1][0]['incomeLevel']['value']
        income_list.append(income_dict)
    count +=1
    
income_list
    
   

result 0 found
result 1 found
result 2 found
result 3 found
result 4 found
result 5 found
result 6 found
result 7 found
result 8 found
result 9 found
result 10 found
result 11 found
result 12 found
result 13 found
result 14 found
result 15 found
result 16 found
result 17 found
result 18 found
result 19 found
result 20 found
result 21 found
result 22 found
result 23 found
result 24 found
result 25 found
result 26 found
result 27 found
result 28 found
result 29 found
result 30 found
result 31 found
result 32 found
result 33 found
result 34 found
result 35 found
result 36 found
result 37 found
result 38 found
result 39 found
result 40 found
result 41 found
result 42 found
result 43 found
result 44 found
result 45 found
result 46 found
result 47 found
result 48 found
result 49 found
result 50 found
result 51 found
result 52 found
result 53 found
result 54 found
result 55 found
result 56 found
result 57 found
result 58 found
result 59 found
result 60 found
result 61 found
result 62 found
re

[{'Country_Name': 'Afghanistan', 'Income_Level': 'Low income'},
 {'Country_Name': 'Albania', 'Income_Level': 'Upper middle income'},
 {'Country_Name': 'Algeria', 'Income_Level': 'Upper middle income'},
 {'Country_Name': 'American Samoa', 'Income_Level': 'Upper middle income'},
 {'Country_Name': 'Andorra', 'Income_Level': 'High income'},
 {'Country_Name': 'Angola', 'Income_Level': 'Lower middle income'},
 {'Country_Name': 'Antigua and Barbuda', 'Income_Level': 'High income'},
 {'Country_Name': 'Argentina', 'Income_Level': 'High income'},
 {'Country_Name': 'Armenia', 'Income_Level': 'Upper middle income'},
 {'Country_Name': 'Aruba', 'Income_Level': 'High income'},
 {'Country_Name': 'Australia', 'Income_Level': 'High income'},
 {'Country_Name': 'Austria', 'Income_Level': 'High income'},
 {'Country_Name': 'Azerbaijan', 'Income_Level': 'Upper middle income'},
 {'Country_Name': 'Bahamas, The', 'Income_Level': 'High income'},
 {'Country_Name': 'Bahrain', 'Income_Level': 'High income'},
 {'Cou